# 🗓️ 12 — DIM_TEMPO (Gold)

- Lê `silver/2018_anonimizado.xlsx` e `silver/2019_anonimizado.xlsx`
- Constrói dimensão de datas usando `DATA`, `DATA_DO_ULTIMO_ATO`, `DATA_DE_ENTRADA_FASE_ATUAL`
- Exporta em `gold/output/dim_tempo.csv`


## 0) Imports

In [ ]:
import pandas as pd
import numpy as np


## 1) Paths robustos

In [ ]:
from pathlib import Path

# ======================================================
# Paths robustos (funciona no VS Code / Jupyter / OneDrive)
# - encontra a raiz do projeto procurando a pasta 'silver'
# ======================================================
CWD = Path().resolve()

def find_project_root(start: Path) -> Path:
    p = start
    for _ in range(12):
        if (p / "silver").exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    raise FileNotFoundError("Não encontrei a pasta 'silver' subindo a árvore. Rode o notebook dentro do repo.")

PROJECT_ROOT = find_project_root(CWD)
SILVER_DIR = PROJECT_ROOT / "silver"
GOLD_DIR = PROJECT_ROOT / "gold"
OUT_DIR = GOLD_DIR / "output"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("📌 CWD:", CWD)
print("📌 PROJECT_ROOT:", PROJECT_ROOT)
print("📌 SILVER_DIR:", SILVER_DIR)
print("📌 OUT_DIR:", OUT_DIR)


## 2) Ler Silver (2018 + 2019)

In [ ]:
import pandas as pd
import numpy as np

INPUT_FILES = [
    SILVER_DIR / "2018_anonimizado.xlsx",
    SILVER_DIR / "2019_anonimizado.xlsx",
]

print("📥 INPUT_FILES:")
for f in INPUT_FILES:
    print(" -", f, "| existe?", f.exists())

dfs = []
for f in INPUT_FILES:
    if not f.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {f}")
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)
print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


## 3) Construir DIM_TEMPO

In [ ]:
date_cols = [c for c in ["DATA", "DATA_DO_ULTIMO_ATO", "DATA_DE_ENTRADA_FASE_ATUAL"] if c in df.columns]
if not date_cols:
    raise KeyError("Não encontrei colunas de data (DATA/DATA_DO_ULTIMO_ATO/DATA_DE_ENTRADA_FASE_ATUAL).")

all_dates = []
for c in date_cols:
    d = pd.to_datetime(df[c], errors="coerce", dayfirst=True).dt.normalize()
    all_dates.append(d)

dates = pd.concat(all_dates, axis=0).dropna().drop_duplicates().sort_values()

dim_tempo = pd.DataFrame({"data": dates})
dim_tempo["id_data"] = dim_tempo["data"].dt.strftime("%Y%m%d").astype(int)
dim_tempo["ano"] = dim_tempo["data"].dt.year
dim_tempo["mes"] = dim_tempo["data"].dt.month
dim_tempo["dia"] = dim_tempo["data"].dt.day
dim_tempo["trimestre"] = dim_tempo["data"].dt.quarter
dim_tempo["semana_ano"] = dim_tempo["data"].dt.isocalendar().week.astype(int)
dim_tempo["dia_semana"] = dim_tempo["data"].dt.dayofweek
dim_tempo["nome_dia"] = dim_tempo["data"].dt.day_name()
dim_tempo["nome_mes"] = dim_tempo["data"].dt.month_name()

print("✅ DIM_TEMPO pronta:", dim_tempo.shape)
dim_tempo.head()


## 4) Exportar

In [ ]:
out_file = OUT_DIR / "dim_tempo.csv"
dim_tempo.to_csv(out_file, index=False, encoding="utf-8")
print("✅ Salvo em:", out_file)
